In [4]:
import json
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from tqdm import tqdm

# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="openai-community/gpt2")

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")



def clean_text(text):
    return text.replace('\n', ' ').replace('\r', ' ').replace('"', "'").strip()

def generate_output(prompt):
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=inputs.shape[1] + 30,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    decoded = tokenizer.decode(outputs[0])[len(prompt):].strip()

    # Only keep the first line and cut off any extra continuation
    clean_output = decoded.split("\n")[0].strip()

    # Optional: Remove unexpected starts like "Example 3:"
    if "Example" in clean_output:
        clean_output = clean_output.split("Example")[0].strip()

    return clean_output



# Load jokes from JSON
with open("reddit_jokes.json", "r", encoding="utf-8") as f:
    jokes_data = json.load(f)

# Output list
joke_pairs = []

# Process each joke
for joke in tqdm(jokes_data):
    title = joke.get("title", "").strip()
    body = joke.get("body", "").strip()
    full_joke = clean_text(f"{title} {body}")
    if len(full_joke) > 140:
        continue  # Skip jokes that are too long

    if not full_joke:
        continue  # skip if empty

    prompt = (
        "Convert the following joke into a neutral, non-humorous statement. Remove any humor, exaggeration, or playful language. "
        "The output should be a serious version of the statement.\n\n"
        "Example 1:\n"
        "Joke: Why don't skeletons fight each other? They don't have the guts.\n"
        "Non-humorous statement: Skeletons lack internal organs, so they cannot fight.\n\n"
        "Example 2:\n"
        "Joke: Why was the math book sad? It had too many problems.\n"
        "Non-humorous statement: The math book contains a lot of problems.\n\n"
        "Now, please convert the following joke into a non-humorous statement:\n"
        f"Joke: {full_joke}\nNon-humorous statement:"
    )
    
    try:
        unfunny = generate_output(prompt)
    except Exception as e:
        print(f"Error with joke: {full_joke[:60]}... | {str(e)}")
        continue

    joke_pairs.append({
        "joke": full_joke,
        "unfunny": unfunny
    })
    print(f"full_joke = {full_joke}")
    print(f"nonjoke = {unfunny}")


# Save to new JSON
with open("joke_unfunny_pairs.json", "w", encoding="utf-8") as f_out:
    json.dump(joke_pairs, f_out, indent=2, ensure_ascii=False)

print(f"Saved {len(joke_pairs)} joke-unfunny pairs to joke_unfunny_pairs.json")


Device set to use cpu
  0%|                                                                            | 1/194553 [00:01<54:34:51,  1.01s/it]

full_joke = I hate how you cant even say black paint anymore Now I have to say 'Leroy can you please paint the fence?'
nonjoke = Now you can't even write 'black paint' in letters.


  0%|                                                                            | 2/194553 [00:01<51:10:48,  1.06it/s]

full_joke = What's the difference between a Jew in Nazi Germany and pizza ? Pizza doesn't scream when you put it in the oven .  I'm so sorry.
nonjoke = The math book contains a lot of problems.


  0%|                                                                            | 6/194553 [00:02<21:07:37,  2.56it/s]

...

full_joke = I need help writing a BDSM joke. I'm having trouble working out all the kinks.
nonjoke = A BDSM joke is a non-objective, non-sexual, non-threatening, non-threatening statement that involves a person's sexual arousal


  0%|                                                                          | 151/194553 [01:36<39:14:09,  1.38it/s]

full_joke = Whats black and screams? Stevie Wonder answering the iron
nonjoke = The math book is about black and screaming.


  0%|                                                                          | 151/194553 [01:36<34:36:35,  1.56it/s]


KeyboardInterrupt: 